# StackingRegressor - 24/11/2023


In [16]:
import pandas as pd
import numpy as np

- Import des données


In [17]:
path = "./../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate
x_predict = pd.read_csv(
    path + "/Xtest.csv", sep=","
)  # features pour la prédiction

- Mise en forme de x_data


In [18]:
x_data = x_data.drop("date", axis=1)  # suppression de la colonne date
x_data = x_data.drop("way", axis=1)  # suppression de la colonne way

# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)


x_data["hour"] = x_data["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
)  # on remplace les Nan de hour par des 0 et les heures par des entiers

# on remplace les Nan des pxqx par 0
x_data[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_data[
    ["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]
].fillna(0)

- Mise en forme de x_predict


In [19]:
x_predict = x_predict.drop("date", axis=1)  # suppression de la colonne date
x_predict = x_predict.drop("way", axis=1)  # suppression de la colonne way

# on tranforme les identifiants de gare en entiers
x_predict["station"] = x_predict["station"].astype("category")
cat_columns = x_predict.select_dtypes(["category"]).columns
x_predict[cat_columns] = x_predict[cat_columns].apply(lambda x: x.cat.codes)


x_predict["hour"] = x_predict["hour"].apply(
    lambda x: 0 if type(x) == float else int(x[:2])
)  # on remplace les Nan de hour par des 0 et les heures par des entiers

# on remplace les Nan des pxqx par 0
x_predict[["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]] = x_predict[
    ["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]
].fillna(0)

- Apprentissage du modèle retenu


In [20]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

In [22]:
XGBR = XGBRegressor(
    n_estimators=4000,
    learning_rate=0.025,
    max_depth=7,
    min_child_weight=8,
    reg_alpha=0.15,
    reg_lambda=0.21,
)
KNN = KNeighborsRegressor(n_neighbors=4, weights="distance", p=1.0)

from sklearn.svm import LinearSVR

estimators = [
    ("knn", KNN),
    ("xgbr", XGBR),
]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearSVR(
        dual="auto",
        random_state=400,
        max_iter=2000,
        epsilon=0.01,
        C=10,
    ),
)
reg.fit(x_data.values, y_data.values)
reg.score(x_data.values, y_data.values)

/home/remiravelli/mambaforge/lib/python3.11/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/remiravelli/mambaforge/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9950825782185497

- Prédiction et export


In [23]:
y_predict = reg.predict(x_predict.values)

In [24]:
from export import export_to_csv

export_to_csv(y_predict, "stackregressor_23_11_24")

Score obtenu : 0.01031962477523498
